In [11]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer, kaggle_dataset_helper

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from ml.models.ensemble import Ensemble

from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common

import ray


import time
import pandas as pd
import pyarrow.fs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# rayer.get_global_cluster()
rayer.get_local_cluster()

<module 'ray' from '/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/ray/__init__.py'>

In [3]:
def use_tokamat_ds():
    df = helper.get_tokamat_dataset()
    df = df.reset_index()

    df = common.label_encode(df)
    df = df.fillna(-1)

    potential_targets = ['WTOT', 'WTH', 'PLTH']
    y = df[potential_targets[0]]

    X = df[df.columns[~df.columns.isin(potential_targets)]]
    X = X.drop(['TOK_ID', 'LCUPDATE', 'DATE', 'NEL', 'ENBI'], axis = 1)

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    return train_test_split(X, y, test_size=0.33)


def use_covid_ds():
    df_X, df_y = helper.get_covid_dataset()
    df_X = df_X.drop(['location'], axis = 1)

    return train_test_split(df_X, df_y, test_size=0.33)


def use_transaction_predictions_ds():
    ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()
    ds_train = common.label_encode(ds_train)
    ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    ds_test = ds_test.fillna(-1)

    df_X = ds_train.loc[:, ds_train.columns != 'target']
    df_y = ds_train['target']

    return train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)



def use_house_pricing_ds():
    ds_train, ds_test = kaggle_dataset_helper.get_house_prices_dataset()
    ds_train = common.label_encode(ds_train)
    ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    ds_test = ds_test.fillna(-1)


    df_X = ds_train.loc[:, ds_train.columns != 'SalePrice']
    df_y = ds_train['SalePrice']

    return train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)


In [4]:
X_train, X_test, y_train, y_test = use_tokamat_ds()

/mnt/c/Users/rwmas/GitHub/xai/python-asd/xai_0.2.0/notebooks/ensemble/../../utils


In [5]:
r2_scoring = make_scorer(score_func=r2_score, greater_is_better=False)

In [ ]:
ens_mdl = Ensemble(   
                                xgb_objective='binary:logistic',  # ["reg:squarederror", "count:poisson", "binary:logistic",  "binary:hinge" ]
                                lgbm_objective='binary',    # https://lightgbm.readthedocs.io/en/latest/Parameters.html
                                pred_class='classification',
                                score_func=None,
                                metric_func=None,
                                list_base_models=[],
                                n_trials=10,          ### common param
                                epochs=15,             ### ANN param
                                boosted_round=10,      ### boosting tree param
                                max_depth=30,          ### boosting tree param
                                max_n_estimators=1500, ### rf param
                                n_estimators=30,       ### bagging param, must be > 10 
                                n_neighbors=30,        ### knn param, must be > 5

                                ensemble_n_estimators=30,  ###  must be > 10
                                ensemble_n_trials=10,
                                timeout=None
                 )

In [28]:
# list_base_models = ['briskbagging', 'briskknn', 'briskxgboost', 'slugxgboost', 'sluglgbm','slugrf']
ens_mdl = Ensemble(   
                                xgb_objective='count:poisson',  # ["reg:squarederror", "count:poisson", "binary:logistic",  "binary:hinge" ]
                                lgbm_objective='poisson',    # https://lightgbm.readthedocs.io/en/latest/Parameters.html
                                pred_class='regression',
                                score_func=None,
                                metric_func=None,
                                list_base_models=['sluglgbm', 'briskxgboost'],
                                n_trials=10,          ### common param
                                epochs=15,             ### ANN param
                                boosted_round=10,      ### boosting tree param
                                max_depth=30,          ### boosting tree param
                                max_n_estimators=1500, ### rf param
                                n_estimators=30,       ### bagging param, must be > 10 
                                n_neighbors=30,        ### knn param, must be > 5

                                ensemble_n_estimators=30,  ###  must be > 10
                                ensemble_n_trials=10,
                                timeout=None
                 )

In [29]:
# threshold, select all model with goodness of fit > threshold
ens_mdl.fetch_models_pll(X_train, X_test, y_train, y_test, threshold=0.1)

2022-12-07T16:13:08CET : INFO : ensemble : fetch_models_pll : 167 : Message : Ensemble: starting discovery process for models [<ml.models.base.tune.slug_lightgbm.SlugLGBM object at 0x7f16c071aa30>, <ml.models.base.tune.brisk_xgboost.BriskXGBoost object at 0x7f16e02aedf0>]


(scheduler +1h23m36s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(scheduler +1h24m12s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(scheduler +1h24m47s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(scheduler +1h25m22s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(sch

In [ ]:
len(ens_mdl.base_models)

2

In [ ]:
ens_mdl.base_model_scores, ens_mdl.scores

([[-1.4034107964169187, -1.4456948604698048],
  [0.999999999999706, 0.9566478490345863]],
 [0.9999949564626046, 0.9569707529868895])

In [22]:
ens_mdl.predict(X_test)

array([1096666.66666667,  755000.        ,  109518.51851852, ...,
        703888.88888889,  259770.37037037,  167096.2962963 ])